**Table of Contents** <br>
1. Load Images and Data Augmentation <br>
2. Create Model <br>
3. Train Model <br>
4. Test Model <br>
5. Submission <br>
6. Conclussion <br>

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from IPython.display import display # Allows the use of display() for DataFrames
from time import time
import matplotlib.pyplot as plt
import seaborn as sns # Plotting library
import keras
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator, img_to_array
from keras.utils import np_utils
from sklearn.datasets import load_files   
from tqdm import tqdm
from collections import Counter


print(os.listdir("../input"))

ModuleNotFoundError: No module named 'tensorflow'

## 1. Data Load

In [3]:
data_train_path = 'C:\Users\ankush.garg.LOHIAGROUP.000\OneDrive - lohiagroup\Documents\Education\EPGPML - C49\4. ML-2\CNN Assignment\CNN_assignment\Skin cancer ISIC The International Skin Imaging Collaboration\Test'
data_valid_path = 'C:\Users\ankush.garg.LOHIAGROUP.000\OneDrive - lohiagroup\Documents\Education\EPGPML - C49\4. ML-2\CNN Assignment\CNN_assignment\Skin cancer ISIC The International Skin Imaging Collaboration\Train'
data_test_path = 'C:\Users\ankush.garg.LOHIAGROUP.000\OneDrive - lohiagroup\Documents\Education\EPGPML - C49\4. ML-2\CNN Assignment\CNN_assignment\Skin cancer ISIC The International Skin Imaging Collaboration\Train'

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (2235027896.py, line 1)

**EDA** <br>

Lets find out how many samples we have for each category.

In [ ]:
# define function to load train, test, and validation datasets
def load_data_raw (path):
    data = load_files(path)
    files = np.array(data['filenames'])
    targets = np_utils.to_categorical(np.array(data['target']), 3)
    
    return files, targets

train_filenames, train_targets = load_data_raw(data_train_path)

In [ ]:
filenames_trimmed = [filename.split('/')[-2] for filename in train_filenames]
classes_count = Counter(filenames_trimmed)

# Plot the classes
plt.bar(classes_count.keys(), classes_count.values(), color=['blue', 'orange', 'green'])

Upsampling function for imbalanced data

Using scikit learn's resample function I will create new samples of the under-represented data.

In [ ]:
def plot_n_samples(filenames):
    filenames_trimmed = [filename.split('/')[-2] for filename in filenames]
    classes_count = Counter(filenames_trimmed)

    # Plot the classes
    plt.bar(classes_count.keys(), classes_count.values(), color=['blue', 'orange', 'green'])

In [ ]:
from sklearn.utils import resample, shuffle

# Choose one of the 3 for the feature_name
feature_names = {0: 'melanoma', 1: 'nevus', 2: 'seborrheic_keratosis'}

def upsample(filenames, targets, feature_name, n_samples = 1372):
    upsample_idx = []
    

    # Find all the indices for nevus
    for i, path in enumerate(filenames):
        # If feature matches, save the index
        if feature_name in path.split('/'):
            upsample_idx.append(i)
    
    # Remove selected features from filenames to add the upsampled after
    new_filenames = [filename for i, filename in enumerate(filenames) if i not in upsample_idx]
    new_targets = [target for i, target in enumerate(targets) if i not in upsample_idx]

    # Upsample
    resampled_x, resampled_y = resample(filenames[upsample_idx], targets[upsample_idx], n_samples=n_samples, random_state=0)

    # Add the upsampled features to new_filenames and new_targets
    new_filenames += list(resampled_x)
    new_targets += list(resampled_y) 
    
    return np.array(new_filenames), np.array(new_targets)
    
# We upsample twice: once for each feature we want upsampled
upsample_train_x, upsample_train_y = upsample(train_filenames, train_targets, feature_names[0])
upsample_train_x, upsample_train_y = upsample(upsample_train_x, upsample_train_y, feature_names[2])

plot_n_samples(upsample_train_x)

In [ ]:
from keras.preprocessing import image   

# Convert the image paths to tensors Manually
def path_to_tensor(img_path):
    # loads RGB image as PIL.Image.Image type
    img = image.load_img(img_path, target_size=(224,224))
    # convert PIL.Image.Image type to 3D tensor with shape (224, 224, 3)
    x = image.img_to_array(img)
    # convert 3D tensor to 4D tensor with shape (1, 224, 224, 3) and return 4D tensor
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)


train_filenames = paths_to_tensor(upsample_train_x)
train_targets = upsample_train_y

In [ ]:
batch_size=60

# Transforms
datagen_train = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.1,  # randomly shift images horizontally 
    height_shift_range=0.1,  # randomly shift images vertically
    horizontal_flip=True)

datagen_valid = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.1,  # randomly shift images horizontally
    height_shift_range=0.1,  # randomly shift images vertically
    horizontal_flip=True)

datagen_test = ImageDataGenerator(
    rescale=1./255)

In [ ]:
# Generators
'''
train_generator = datagen_train.flow_from_directory(
        data_train_path,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical')
'''

train_generator = datagen_train.flow(train_filenames, train_targets, batch_size=batch_size)

valid_generator = datagen_valid.flow_from_directory(
        data_valid_path,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False)

test_generator = datagen_test.flow_from_directory(
        data_test_path,
        target_size=(224, 224),
        batch_size=1,
        class_mode='categorical',
        shuffle=False)

In [ ]:
num_train = len(train_filenames)
num_valid = len(valid_generator.filenames)
num_test = len(test_generator.filenames)

print(num_train, num_valid, num_test)

Get the class indices <br>

To get the label we find the index of the 1 in the one hot encoded vector which should match the index in a dictionary. Eg: <br>

- label = [1,0,0] ---> class_index = 0
- label = [0,1,0] ---> class_index = 1
- label = [0,0,1] ---> class_index = 2

In [ ]:
# Class name to the index
#class_2_indices = train_generator.class_indices
class_2_indices = {'melanoma': 0, 'nevus': 1, 'seborrheic_keratoses': 2}
print("Class to index:", class_2_indices)

# Reverse dict with the class index to the class name
indices_2_class = {v: k for k, v in class_2_indices.items()}
print("Index to class:", indices_2_class)

In [ ]:
 # Lets have a look at some of our images
images, labels = train_generator.next()

fig = plt.figure(figsize=(20,10))
fig.subplots_adjust(wspace=0.2, hspace=0.4)

# Lets show the first 32 images of a batch
for i, img in enumerate(images[:32]):
    ax = fig.add_subplot(4, 8, i + 1, xticks=[], yticks=[])
    ax.imshow(img)
    image_idx = np.argmax(labels[i])
    ax.set(title=indices_2_class[image_idx])

## 2. Create the model

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization, GlobalAveragePooling2D
from keras.applications import ResNet50
from keras.models import Model
from keras_tqdm import TQDMNotebookCallback

base_model = ResNet50(weights='imagenet', include_top=False)

# add a global spatial average pooling layer
x = base_model.output
x = GlobalAveragePooling2D()(x)
# x = MaxAveragePooling2D()(x)
# let's add a fully-connected layer
x = Dense(1024, activation='elu')(x)
x = Dropout(0.95)(x)
# and a logistic layer
predictions = Dense(3, activation='softmax')(x)

# this is the model we will train
model = Model(inputs=base_model.input, outputs=predictions)

# first: train only the top layers (which were randomly initialized)
for layer in base_model.layers:
    layer.trainable = True

#model.summary()

In [ ]:
from keras.optimizers import Adam

# compile the model (should be done *after* setting layers to non-trainable)
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy',
             metrics=['accuracy'])

## 3. Train the model

Assign weights to imbalanced classes

The dataset shows an imbalance. By assigning bigger weights to the misrepresented classes in the dataset we will help to correct this issue.

In [ ]:
from sklearn.utils import class_weight

# Convert one hot encoded labels to ints
train_targets_classes = [np.argmax(label) for label in train_targets]

# Compute the weights
class_weights = class_weight.compute_class_weight('balanced',
                                                  np.unique(train_targets_classes),
                                                  train_targets_classes)

class_weights_dict = dict(enumerate(class_weights))
print(class_weights_dict)

## 4. Train the model

In [ ]:
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping

# train the model
checkpointer = ModelCheckpoint(filepath='aug_model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)

scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1,
                              patience=5, min_lr=1e-8, verbose=1)

early_stopper = EarlyStopping(monitor='val_loss', patience=10,
                              verbose=0, restore_best_weights=True)

history = model.fit_generator(train_generator,
                    class_weight= class_weights_dict,
                    steps_per_epoch=num_train//batch_size,
                    epochs=40,
                    verbose=0,
                    callbacks=[checkpointer, scheduler, TQDMNotebookCallback(), early_stopper],
                    validation_data=valid_generator,
                    validation_steps=num_valid//batch_size)

## 5. Test model

Load the model with the best Validation Accuracy

In [ ]:
# load the weights that yielded the best validation accuracy
model.load_weights('aug_model.weights.best.hdf5')

In [ ]:
score = model.evaluate_generator(test_generator, steps=num_test//1, verbose=1)
print('\n', 'Test accuracy:', score[1])

Predictions for the test data

- task_1: the model's predicted probability that the image depicts melanoma
- task_2: the model's predicted probability that the image depicts seborrheic keratosis

In [ ]:
predictions = model.predict_generator(test_generator, steps=num_test)

task_1 = pd.DataFrame(data=[desease[0] for desease in predictions])
task_2 = pd.DataFrame(data=[desease[2] for desease in predictions])

RocAuc Score

In [ ]:
from sklearn.metrics import roc_auc_score, accuracy_score

ground_truth = pd.read_csv("../input/udacitydermatologistai/repository/udacity-dermatologist-ai-2ec0ca9/ground_truth.csv")
labels = np_utils.to_categorical(np.array(test_generator.classes), 3)

roc_auc_all = roc_auc_score(labels, predictions)
roc_auc_task_1 = roc_auc_score(ground_truth['task_1'], task_1)
roc_auc_task_2 = roc_auc_score(ground_truth['task_2'], task_2)

print('Roc auc score for all data is: {}'.format(roc_auc_all))
print('Roc auc score for task 1 is: {}'.format(roc_auc_task_1))
print('Roc auc score for task 2 is: {}'.format(roc_auc_task_2))

Lets visualize some of our predictions

In [4]:
test_filenames, test_targets = load_data_raw(data_test_path)

NameError: name 'load_data_raw' is not defined

In [ ]:
def plot_prediction(img_file, img_target):

    img = image.load_img(img_file, target_size=(224,224))
    img = image.img_to_array(img)/255
    img_expand = np.expand_dims(img, axis=0)
    
    # Make a prediction
    prediction = model.predict(img_expand, steps=1)
    image_idx = np.argmax(prediction[0])
    prediction_string = indices_2_class[image_idx]
    
    # Get the real label's name
    label_idx = np.argmax(img_target)
    real_label = indices_2_class[label_idx]
    
    # Plot predictions
    title = "Prediction: {}\nReal: {}".format(prediction_string, real_label)
    
    plt.imshow(img)
    plt.title(title)
    
    pred_df = pd.DataFrame({'Cancer type':['melanoma', 'nevus', 'seborrheic keratosis'], 'val':prediction[0]})
    ax = pred_df.plot.barh(x='Cancer type', y='val', title="Predictions", grid=True)
    
random_index = np.random.randint(0, len(test_generator.filenames))
plot_prediction(test_filenames[random_index], test_targets[random_index])

Performance curves

In [ ]:
plts, (ax1, ax2) = plt.subplots(1,2, figsize=(20,5))

# summarize history for accuracy
ax1.plot(history.history['acc'])
ax1.plot(history.history['val_acc'])
ax1.set_title('model accuracy')
ax1.set(xlabel='epoch', ylabel='accuracy')
ax1.legend(['train', 'val'], loc='upper left')

ax2.plot(history.history['loss'])
ax2.plot(history.history['val_loss'])
ax2.set_title('model loss')
ax2.set(xlabel='epoch', ylabel='loss')
ax2.legend(['train', 'val'], loc='upper left')

## 5. Submission

In [ ]:
submission = pd.read_csv("../input/udacitydermatologistai/repository/udacity-dermatologist-ai-2ec0ca9/sample_predictions.csv")
submission['task_1'] = task_1
submission['task_2'] = task_2
submission.to_csv("submission_dermatologist.csv", index=False)
display(submission.head())

## 6. Conclusion

Due to the nature of the problem, it is very important that we make no mistake when identifying a patient as not sick. We prefer to send someone who is not sick for more tests, rather than someone who is sick home. This is what we know as recall, and we want a value for recall as close to 1 as possible. To improve recall what we can do is set a lower threshold to idenfify someone as sick or not sick, therefore only the pateients with a really low probability of melanoma will be let go, any others with a higher predicted chance will be held for more tests.

Confusion matrix

In [ ]:
from sklearn.metrics import confusion_matrix

# Confusion matrix for all classes
y_true = test_generator.classes
y_pred = [np.argmax(x) for x in predictions]

labels = ["melanoma", "nevus", "keratoses"]
cm = confusion_matrix(y_true, y_pred)
cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] # Normalize confusion matrix
ax = sns.heatmap(cm, annot=True)
ax.xaxis.set_ticklabels(labels)
ax.yaxis.set_ticklabels(labels)

In [ ]:
def plot_distribution(pred_target_y, filenames):
    melanoma_idx = []
      
    # Find all the indices for nevus
    for i, path in enumerate(filenames):
        # If feature matches, save the index
        if 'melanoma' in path.split('/'):
            melanoma_idx.append(i)
            
    bening_preds = [pred for i, pred in enumerate(pred_target_y) if i not in melanoma_idx]
    malignant_preds = [pred for i, pred in enumerate(pred_target_y) if i in melanoma_idx]
    
    fig, ax = plt.subplots(1,1,figsize=(15,6))
    
    ax.set_title('Malignant vs. Bening')
    sns.distplot(bening_preds, hist=True, kde=True, label="Benign", bins=35)
    sns.distplot(malignant_preds, hist=True, kde=True, label="Malignant", bins=35, axlabel="Probability Malignant")
    ax.legend()
    ax.xaxis.set_ticks(np.arange(0, 1.1, 0.1))

plot_distribution(task_1.values, test_generator.filenames)

__Insights__

We can see the a little the separation between malignant and bening lesions. This plot could be used to find the best threshold that maximizes the amount of people that have malignant lesions.

- The model predicts that about 50% of the malignant lesions are bening (Really close to 0%)
- The model predicts correctly 50% of the malignant lesions as malignant (as we can see in the confusion matrix)
- In the event that we had to choose a threshold (this is dangerous because a lot of our malignant lesions are very near the 0), we could say that a "good" threshold is around 0.05, where it identifies a lot of malignant lesions (more than 50% at least) and leaves back most benign lesions (more than 90%)